In [9]:
import os
current_directory = os.getcwd()
folder_name = os.path.basename(current_directory)
number = folder_name

In [10]:
import platform
platform_name = platform.system()

In [11]:
CONFIG = {
    'test_path': 'test.csv',
    'train_path': 'train.csv',

    'DEVICE' : 'cuda',
    'SEED' : 42,
}
if platform_name == 'Windows':
    CONFIG['data_main'] = 'C:/Users/Николай/PycharmProjects/CIBMTR/D.Data/main/'
    CONFIG['model_path'] = f"{number}_model.pkl"
else:
    CONFIG['data_main'] = '/kaggle/input/equity-post-HCT-survival-predictions/'
    CONFIG['model_path'] = "/kaggle/input/2.1.18_model.pkl/other/default/1/2.1.18_model.pkl"

In [12]:

import pandas as pd
import joblib 

In [13]:
data_exp = pd.read_csv(f"{CONFIG['data_main']}data_dictionary.csv")

In [14]:
train = pd.read_csv(f"{CONFIG['data_main']}{CONFIG['train_path']}")
train[['sex_1', 'sex_2']] = train['sex_match'].str.split('-', expand=True)
train.drop(columns=['ID'], inplace=True)

num_columns = []
for i, row in data_exp.iterrows():
    if row['type'] == 'Numerical':
        num_columns.append(row['variable'])
num_columns.append('efs')
cat_columns = [col for col in train.columns if col not in num_columns]
num_columns.remove('efs')
num_columns.remove('efs_time')
na_num_columns = []
for col in num_columns:
    train[f"{col}_nan"] = train[col].isna().astype(int)
    train[f"{col}_nan"] = train[col].isna().astype(int)
    na_num_columns.append(f"{col}_nan")
    
num_columns.extend(na_num_columns)

for col in train[cat_columns].columns:
    train[col] = train[col].astype(str)
    j_ch = ',[]{}:"\\<'  # набор символов для удаления
    for ch in j_ch:
        train[col] = train[col].apply(lambda x: str(x).replace(ch, ''))
train_one_hot = pd.get_dummies(train[cat_columns])
cat_columns = list(train_one_hot.columns)
train_one_hot = pd.concat([train_one_hot, train[num_columns]], axis=1)

In [15]:
test = pd.read_csv(f"{CONFIG['data_main']}test.csv")
test[['sex_1', 'sex_2']] = test['sex_match'].str.split('-', expand=True)
num_columns = []
for i, row in data_exp.iterrows():
    if row['type'] == 'Numerical':
        num_columns.append(row['variable'])
num_columns.append('efs')
cat_columns = [col for col in test.columns if col not in num_columns]
num_columns.remove('efs')
num_columns.remove('efs_time')

na_num_columns = []
for col in num_columns:
    test[f"{col}_nan"] = test[col].isna().astype(int)
    test[f"{col}_nan"] = test[col].isna().astype(int)
    na_num_columns.append(f"{col}_nan")
    
num_columns.extend(na_num_columns)

for col in test[cat_columns].columns:
    test[col] = test[col].astype(str)
    j_ch = ',[]{}:"\\<' 
    for ch in j_ch:
        test[col] = test[col].apply(lambda x: str(x).replace(ch, ''))
test_one_hot = pd.get_dummies(test[cat_columns])
test_one_hot = pd.concat([test_one_hot, test[num_columns]], axis=1)

missing_columns = set(train_one_hot.columns) - set(test_one_hot.columns)
for col in missing_columns:
    test_one_hot[col] = False
    
drop_cols = ['hla_match_dqb1_low_nan',
 'karnofsky_score_nan',
 'hla_low_res_8_nan',
 'comorbidity_score_nan',
 'hla_match_b_high_nan',
 'hla_match_a_low_nan',
 'hla_match_drb1_high_nan',
 'age_at_hct_nan',
 'hla_match_b_low_nan',
 'donor_age_nan',
 'hla_match_a_high_nan',
 'year_hct_nan',
 'hla_high_res_6_nan',
 'hla_match_drb1_low_nan',
 'hla_match_c_low_nan',
 'hla_nmdp_6_nan',
 'hla_match_dqb1_high_nan',
 'hla_high_res_10_nan',
 'hla_low_res_6_nan',
 'hla_high_res_8_nan',
 'hla_match_c_high_nan',
 'gvhd_proph_FK+- others(not MMFMTX)',
 'graft_type_Peripheral blood',
 'prod_type_PB',
 'hla_low_res_10_nan']
drop_cols_2 = ['sex_1_F',
 'sex_2_F',
 'tce_match_HvG non-permissive',
 'prim_disease_hct_PCD',
 'dri_score_Very high',
 'tce_imm_match_H/B',
 'pulm_severe_nan',
 'cyto_score_Other',
 'renal_issue_No',
 'diabetes_nan',
 'vent_hist_nan',
 'sex_1_M',
 'renal_issue_nan',
 'dri_score_nan',
 'hepatic_severe_Not done',
 'dri_score_Intermediate - TED AML case missing cytogenetics',
 'gvhd_proph_nan',
 'tce_div_match_Bi-directional non-permissive',
 'prim_disease_hct_AI',
 'peptic_ulcer_nan',
 'gvhd_proph_Other GVHD Prophylaxis',
 'peptic_ulcer_Yes',
 'rheum_issue_nan',
 'psych_disturb_nan',
 'psych_disturb_Not done',
 'sex_2_M',
 'ethnicity_Non-resident of the U.S.',
 'cardiac_Not done',
 'rheum_issue_Not done',
 'dri_score_N/A - disease not classifiable',
 'pulm_severe_Not done',
 'conditioning_intensity_No drugs reported',
 'cyto_score_Normal',
 'hepatic_severe_nan',
 'obesity_Not done',
 'tbi_status_TBI +- Other -cGy unknown dose',
 'pulm_moderate_Not done',
 'tbi_status_TBI +- Other -cGy single',
 'renal_issue_Yes',
 'prim_disease_hct_Other leukemia',
 'tbi_status_TBI +- Other -cGy fractionated',
 'gvhd_proph_CSA + MTX +- others(not MMFFK)',
 'prim_disease_hct_Solid tumor',
 'gvhd_proph_TDEPLETION +- other',
 'prim_disease_hct_HD',
 'gvhd_proph_CDselect alone',
 'gvhd_proph_No GvHD Prophylaxis',
 'renal_issue_Not done',
 'hepatic_mild_Not done',
 'sex_match_nan',
 'prim_disease_hct_Other acute leukemia',
 'prim_disease_hct_CML',
 'gvhd_proph_CSA alone',
 'sex_1_nan',
 'tbi_status_TBI +- Other unknown dose',
 'prim_disease_hct_IMD',
 'gvhd_proph_Parent Q = yes but no agent',
 'conditioning_intensity_N/A F(pre-TED) not submitted',
 'tce_imm_match_P/H',
 'sex_2_nan',
 'cyto_score_Not tested',
 'gvhd_proph_CDselect +- other',
 'dri_score_Missing disease status',
 'gvhd_proph_CSA +- others(not FKMMFMTX)',
 'tce_imm_match_P/G',
 'tce_imm_match_P/B']
drop_cols.extend(drop_cols_2)
train_one_hot.drop(columns=drop_cols, inplace=True)
test_one_hot.drop(columns=drop_cols, inplace=True)

C:\Users\Николай\AppData\Local\Temp\ipykernel_12844\1408835055.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_one_hot[col] = False
C:\Users\Николай\AppData\Local\Temp\ipykernel_12844\1408835055.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_one_hot[col] = False
C:\Users\Николай\AppData\Local\Temp\ipykernel_12844\1408835055.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at o

In [16]:
test_one_hot = test_one_hot[train_one_hot.columns]

In [17]:
final_cat = joblib.load(CONFIG['model_path'])

In [18]:
preds_test = final_cat.predict(test_one_hot)

In [19]:
y_pred = test[['ID']].copy(deep=True)
y_pred["prediction"] = preds_test
y_pred.to_csv('submission.csv', index=False)

In [20]:
y_pred

,ID,prediction
0,28800,0.441868
1,28801,0.636648
2,28802,0.382843
